In [ ]:
!pip install flask-cors
from flask_cors import CORS



In [ ]:
!pip install flask pyngrok sentence-transformers faiss-gpu pandas tqdm
!ngrok config add-authtoken 2lKfqzJoqD6Ept2ReqnWHG9xQeR_5DZYZxia5Mq7Rm39WyvaR


In [ ]:
# Frontend Code: Run this after the backend
from flask import Flask, render_template_string
from pyngrok import ngrok

# Flask app initialization
app = Flask(__name__)
# CORS(app)

# HTML template for the frontend
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Weaver AI</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600&display=swap" rel="stylesheet">
  <style>
    body { margin: 0; padding: 0; background-color: #f0f2f5; font-family: 'Inter', sans-serif; display: flex; justify-content: center; align-items: center; height: 100vh; }
    .chat-container { width: 90%; max-width: 600px; height: 80vh; background-color: #ffffff; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); border-radius: 15px; display: flex; flex-direction: column; overflow: hidden; }
    .chat-header { background-color: #4a4e69; color: #ffffff; text-align: center; padding: 15px; font-size: 20px; font-weight: 600; }
    .messages { flex-grow: 1; padding: 15px; overflow-y: auto; background-color: #fafafa; }
    .message { display: flex; align-items: center; margin: 10px 0; }
    .message.bot .text { background-color: #eaeaea; border-radius: 10px; padding: 10px 15px; margin-left: 10px; max-width: 70%; }
    .message.user { justify-content: flex-end; }
    .message.user .text { background-color: #dcedf7; border-radius: 10px; padding: 10px 15px; margin-right: 10px; max-width: 70%; }
    .input-container { display: flex; padding: 10px; border-top: 1px solid #e0e0e0; background-color: #ffffff; }
    .input-container input { flex-grow: 1; padding: 10px; border: 1px solid #e0e0e0; border-radius: 25px; margin-right: 10px; }
    .input-container button { background-color: #4a4e69; color: white; border: none; width: 45px; height: 45px; border-radius: 50%; cursor: pointer; margin-left: 5px; }
     .active {
    background-color: #ff6b6b !important;
    animation: pulse 1s infinite;
    .message.bot .text {
  background-color: #e0e4e7;
  color: #333;
  padding: 12px 20px;
  margin-left: 10px;
  border-radius: 25px;
  border: 1px solid #b0b8bc;
  max-width: 70%;
  display: inline-block;
  text-align: center;
  transition: background-color 0.3s, box-shadow 0.3s;
  cursor: pointer;
}

.message.bot .text:hover {
  background-color: #cfd6db;
  box-shadow: 0 2px 8px rgba(0, 0, 0, 0.2);
}

.message.user .text {
  background-color: #dcedf7;
  border-radius: 25px;
  padding: 12px 20px;
  margin-right: 10px;
  max-width: 70%;
  text-align: center;
}
  }

  @keyframes pulse {
    0% { box-shadow: 0 0 5px rgba(255, 107, 107, 0.8); }
    50% { box-shadow: 0 0 20px rgba(255, 107, 107, 0.8); }
    100% { box-shadow: 0 0 5px rgba(255, 107, 107, 0.8); }
  }
  </style>
</head>
<body>

  <div class="chat-container">
    <div class="chat-header">Weaver AI</div>
    <div class="messages" id="chat"></div>
    <div class="input-container">
      <input type="text" id="userInput" placeholder="Ask something..." onkeydown="handleKeyPress(event)">
      <button onclick="sendMessage()">Send</button>
      <button onclick="startRecording()">🎤</button>
    </div>
  </div>

  <script>
    const chatContainer = document.getElementById('chat');
    const userInput = document.getElementById('userInput');

    function addMessage(content, className) {
      const message = document.createElement('div');
      message.className = `message ${className}`;
      message.innerHTML = `<div class="text">${content}</div>`;
      chatContainer.appendChild(message);
      chatContainer.scrollTop = chatContainer.scrollHeight;
    }

    async function sendMessage() {
      const message = userInput.value;
      if (message) {
        addMessage(message, 'user');
        userInput.value = '';

        try {
          const response = await fetch('https://3912-104-196-242-58.ngrok-free.app/retrieve', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ query: message })
          });

          const result = await response.json();
          displayResponse(result);
        } catch (error) {
          addMessage('Error communicating with backend.', 'bot');
        }
      }
    }

    function displayResponse(results) {
      let content = '<ul>';
      results.forEach(item => {
        content += `<li><a href="${item.Hyperlink}" target="_blank">${item.Title}</a></li>`;
      });
      content += '</ul>';
      addMessage(content, 'bot');
    }

    function handleKeyPress(event) {
      if (event.key === 'Enter') {
        sendMessage();
      }
    }

    async function startRecording() {
  const micButton = document.querySelector('button[onclick="startRecording()"]');

  try {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream, { mimeType: 'audio/webm' });
    const audioChunks = [];

    // Highlight the button while recording
    micButton.classList.add('active');

    mediaRecorder.ondataavailable = (event) => {
      audioChunks.push(event.data);
    };

    mediaRecorder.onstop = async () => {
      // Remove button highlight when recording stops
      micButton.classList.remove('active');

      const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });
      const audioFile = new File([audioBlob], 'audio.wav', { type: 'audio/wav' });
      const formData = new FormData();
      formData.append('file', audioFile);
      formData.append('model', 'saaras:v1');
      formData.append('prompt', 'Convert the speech to English.');

      const response = await fetch('https://api.sarvam.ai/speech-to-text-translate', {
        method: 'POST',
        headers: { 'api-subscription-key': '38d891a7-0c23-4448-92d9-fe15dbb5578a' },
        body: formData
      });

      const result = await response.json();
      userInput.value = result.transcript || 'Could not transcribe audio';
    };

    mediaRecorder.start();

    // Stop recording after 5 seconds
    setTimeout(() => {
      mediaRecorder.stop();
    }, 5000);
  } catch (error) {
    console.error('Error with recording:', error);
    micButton.classList.remove('active');  // Remove highlight if there's an error
    addMessage('Could not access microphone.', 'bot');
  }
}
  </script>

</body>
</html>
"""

# Route to render the frontend
@app.route('/')
def home():
    return render_template_string(html_content)

# Expose the backend function directly for JS to call
@app.route('/retrieve', methods=['POST'])
def retrieve():
    query = request.json.get('query')
    result = retrieve_top_n(query, 5, model, index, data)
    return {"results": result}

# Ngrok public URL configuration
public_url = ngrok.connect(5001)
print(f"Access your app at: {public_url}")

# Run the Flask app
if __name__ == "__main__":
    app.run(port=5001)


Access your app at: NgrokTunnel: "https://2159-34-74-136-42.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Oct/2024 05:33:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Oct/2024 05:33:03] "GET /favicon.ico HTTP/1.1" 404 -
